In [2]:
import torch
%load_ext autoreload
%autoreload 2

In [3]:
from model.threedepn import ThreeDEPNDecoder
from util.model import summarize_model

threedepn = ThreeDEPNDecoder()
print(summarize_model(threedepn))

   | Name         | Type             | Params  
-----------------------------------------------------
0  | bottleneck   | Sequential       | 197376  
1  | bottleneck.0 | Linear           | 65792   
2  | bottleneck.1 | ReLU             | 0       
3  | bottleneck.2 | Linear           | 131584  
4  | bottleneck.3 | ReLU             | 0       
5  | decoder1     | Sequential       | 8389376 
6  | decoder1.0   | ConvTranspose3d  | 8388864 
7  | decoder1.1   | BatchNorm3d      | 512     
8  | decoder1.2   | ReLU             | 0       
9  | decoder2     | Sequential       | 2097536 
10 | decoder2.0   | ConvTranspose3d  | 2097280 
11 | decoder2.1   | BatchNorm3d      | 256     
12 | decoder2.2   | ReLU             | 0       
13 | decoder3     | Sequential       | 524480  
14 | decoder3.0   | ConvTranspose3d  | 524352  
15 | decoder3.1   | BatchNorm3d      | 128     
16 | decoder3.2   | ReLU             | 0       
17 | decoder4     | Sequential       | 4097    
18 | decoder4.0   | ConvTranspose3

In [4]:
from data.shapenet import ShapeNet

# Create a dataset with train split
train_dataset = ShapeNet('train', filter_class='lamp')
val_dataset = ShapeNet('val', filter_class='lamp')
test_dataset = ShapeNet('test', filter_class='lamp')

print(f'Length of train set: {len(train_dataset)}')  # expected output: 153540
print(f'Length of val set: {len(val_dataset)}')  # expected output: 153540
print(f'Length of test set: {len(test_dataset)}')  # expected output: 64

Length of train set: 1854
Length of val set: 232
Length of test set: 232


In [5]:
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes

sample = test_dataset[231]
print(f'Target DF: {sample["target_df"].shape}')  # expected output: (32, 32, 32)
print(f'Target DF: {type(sample["target_df"])}')  # expected output: <class 'numpy.ndarray'>

input_mesh = marching_cubes(sample['target_df'], level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

Target DF: (32, 32, 32)
Target DF: <class 'numpy.ndarray'>


Output()

In [6]:
##################
#                #
#    TRAINING    #
#                #
##################

In [7]:
# AIRPLANE AD
from scripts import train
config = {
    'experiment_name': 'airplane_ad',
    'device': 'cuda:0',
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 1000,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : False,
    'test': False,
    'kl_weight': 0.03,
    'kl_weight_increase_every_epochs': 100,
    'kl_weight_increase_value': 0.0,
    'iou_every_epoch': 50,
    'resume_ckpt': None,
    'filter_class': 'airplane',
    'decoder_var' : False
}
train.main(config)

Using device: cuda:0
Data length 3236
Training params: 2
[001/00048] train_loss: 0.106201 kl_loss: 0.000000 normal_loss: 0.106201
[003/00046] train_loss: 0.042423 kl_loss: 0.000000 normal_loss: 0.042423
[005/00044] train_loss: 0.035614 kl_loss: 0.000000 normal_loss: 0.035614
[007/00042] train_loss: 0.031330 kl_loss: 0.000000 normal_loss: 0.031330
[009/00040] train_loss: 0.029408 kl_loss: 0.000000 normal_loss: 0.029408
[011/00038] train_loss: 0.028273 kl_loss: 0.000000 normal_loss: 0.028273
[013/00036] train_loss: 0.025522 kl_loss: 0.000000 normal_loss: 0.025522
[015/00034] train_loss: 0.024653 kl_loss: 0.000000 normal_loss: 0.024653
[017/00032] train_loss: 0.025657 kl_loss: 0.000000 normal_loss: 0.025657
[019/00030] train_loss: 0.022715 kl_loss: 0.000000 normal_loss: 0.022715
[021/00028] train_loss: 0.021348 kl_loss: 0.000000 normal_loss: 0.021348
[023/00026] train_loss: 0.020369 kl_loss: 0.000000 normal_loss: 0.020369
[025/00024] train_loss: 0.024364 kl_loss: 0.000000 normal_loss: 0.0

In [44]:
# TABLE AD
from scripts import train
config = {
    'experiment_name': 'table_ad',
    'device': 'cuda:0',
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 1000,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : False,
    'test': False,
    'kl_weight': 0.03,
    'kl_weight_increase_every_epochs': 100,
    'kl_weight_increase_value': 0.0,
    'iou_every_epoch': 50,
    'resume_ckpt': None,
    'filter_class': 'table',
    'decoder_var' : False
}
train.main(config)

Using device: cuda:0
Data length 4800
Training params: 2
[001/00024] train_loss: 0.168495 kl_loss: 0.000000 normal_loss: 0.168495
[002/00049] train_loss: 0.117954 kl_loss: 0.000000 normal_loss: 0.117954
[003/00074] train_loss: 0.109562 kl_loss: 0.000000 normal_loss: 0.109562
[005/00024] train_loss: 0.105227 kl_loss: 0.000000 normal_loss: 0.105227
[006/00049] train_loss: 0.097987 kl_loss: 0.000000 normal_loss: 0.097987
[007/00074] train_loss: 0.095581 kl_loss: 0.000000 normal_loss: 0.095581
[009/00024] train_loss: 0.091399 kl_loss: 0.000000 normal_loss: 0.091399
[010/00049] train_loss: 0.086116 kl_loss: 0.000000 normal_loss: 0.086116
[011/00074] train_loss: 0.082822 kl_loss: 0.000000 normal_loss: 0.082822
[013/00024] train_loss: 0.076092 kl_loss: 0.000000 normal_loss: 0.076092
[014/00049] train_loss: 0.073706 kl_loss: 0.000000 normal_loss: 0.073706
[015/00074] train_loss: 0.068664 kl_loss: 0.000000 normal_loss: 0.068664
[017/00024] train_loss: 0.065321 kl_loss: 0.000000 normal_loss: 0.0

In [43]:
# CHAIR AD
from scripts import train
config = {
    'experiment_name': 'chair_ad',
    'device': 'cuda:0',
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 1000,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : False,
    'test': False,
    'kl_weight': 0.03,
    'kl_weight_increase_every_epochs': 100,
    'kl_weight_increase_value': 0.0,
    'iou_every_epoch': 50,
    'resume_ckpt': None,
    'filter_class': 'chair',
    'decoder_var' : False
}
train.main(config)

Using device: cuda:0
Data length 4800
Training params: 2
[001/00024] train_loss: 0.164451 kl_loss: 0.000000 normal_loss: 0.164451
[002/00049] train_loss: 0.116169 kl_loss: 0.000000 normal_loss: 0.116169
[003/00074] train_loss: 0.113166 kl_loss: 0.000000 normal_loss: 0.113166
[005/00024] train_loss: 0.106087 kl_loss: 0.000000 normal_loss: 0.106087
[006/00049] train_loss: 0.100100 kl_loss: 0.000000 normal_loss: 0.100100
[007/00074] train_loss: 0.095202 kl_loss: 0.000000 normal_loss: 0.095202
[009/00024] train_loss: 0.089955 kl_loss: 0.000000 normal_loss: 0.089955
[010/00049] train_loss: 0.085373 kl_loss: 0.000000 normal_loss: 0.085373
[011/00074] train_loss: 0.082074 kl_loss: 0.000000 normal_loss: 0.082074
[013/00024] train_loss: 0.077538 kl_loss: 0.000000 normal_loss: 0.077538
[014/00049] train_loss: 0.075404 kl_loss: 0.000000 normal_loss: 0.075404
[015/00074] train_loss: 0.071963 kl_loss: 0.000000 normal_loss: 0.071963
[017/00024] train_loss: 0.069499 kl_loss: 0.000000 normal_loss: 0.0

In [ ]:
# TABLE VAD
from scripts import train
config = {
    'experiment_name': 'table_vad',
    'device': 'cuda:0',
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 1000,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'kl_weight_increase_every_epochs': 100,
    'kl_weight_increase_value': 0.01,
    'resume_ckpt': 'table_vad',
    'filter_class': 'table',
    'decoder_var' : True
}
train.main(config)

In [ ]:
# AIRPLANE VAD
from scripts import train
config = {
    'experiment_name': 'airplane_vad',
    'device': 'cuda:0',
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 1000,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'kl_weight_increase_every_epochs': 100,
    'kl_weight_increase_value': 0.0,
    'resume_ckpt': 'airplane_vad',
    'filter_class': 'airplane',
    'decoder_var' : True
}
train.main(config)

In [166]:
# CHAIR VAD
from scripts import train
config = {
    'experiment_name': 'chair_vad',
    'device': 'cuda:0',
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 1000,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'kl_weight_increase_every_epochs': 100,
    'kl_weight_increase_value': 0.0,
    'mmd_every_epoch': 50,
    'tmd_every_epoch': 50,
    'iou_every_epoch': 10,
    'resume_ckpt': None,
    'filter_class': 'chair',
    'decoder_var' : True
}
train.main(config)

Using device: cuda:0
Data length 4800
Training params: 3
[001/00024] train_loss: 0.178638 kl_loss: 0.492069 normal_loss: 0.163876
[002/00049] train_loss: 0.133184 kl_loss: 0.440588 normal_loss: 0.119966
[003/00074] train_loss: 0.126972 kl_loss: 0.387870 normal_loss: 0.115336
[005/00024] train_loss: 0.122300 kl_loss: 0.344194 normal_loss: 0.111974
[006/00049] train_loss: 0.117572 kl_loss: 0.326850 normal_loss: 0.107766
[007/00074] train_loss: 0.114495 kl_loss: 0.316428 normal_loss: 0.105002
[009/00024] train_loss: 0.109007 kl_loss: 0.302268 normal_loss: 0.099939
[010/00049] train_loss: 0.107026 kl_loss: 0.290253 normal_loss: 0.098318
[011/00074] train_loss: 0.105014 kl_loss: 0.278051 normal_loss: 0.096672
[013/00024] train_loss: 0.103401 kl_loss: 0.266038 normal_loss: 0.095420
[014/00049] train_loss: 0.099887 kl_loss: 0.255598 normal_loss: 0.092219
[015/00074] train_loss: 0.098646 kl_loss: 0.247489 normal_loss: 0.091221
[017/00024] train_loss: 0.097429 kl_loss: 0.241518 normal_loss: 0.0

In [216]:
# CHAIR VAD
from scripts import train
config = {
    'experiment_name': 'chair_vad_0.05kl',
    'device': 'cuda:0',
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 1000,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.05,
    'kl_weight_increase_every_epochs': 100,
    'kl_weight_increase_value': 0.0,
    'mmd_every_epoch': 50,
    'tmd_every_epoch': 50,
    'iou_every_epoch': 10,
    'resume_ckpt': None,
    'filter_class': 'chair',
    'decoder_var' : True
}
train.main(config)

Using device: cuda:0
Data length 4800
Training params: 3
[001/00024] train_loss: 0.209842 kl_loss: 0.489154 normal_loss: 0.160927
[002/00049] train_loss: 0.160607 kl_loss: 0.429894 normal_loss: 0.117618
[003/00074] train_loss: 0.150132 kl_loss: 0.368432 normal_loss: 0.113288
[005/00024] train_loss: 0.142441 kl_loss: 0.308028 normal_loss: 0.111638
[006/00049] train_loss: 0.137221 kl_loss: 0.266177 normal_loss: 0.110604
[007/00074] train_loss: 0.131735 kl_loss: 0.236801 normal_loss: 0.108055
[009/00024] train_loss: 0.128022 kl_loss: 0.216909 normal_loss: 0.106331
[010/00049] train_loss: 0.123229 kl_loss: 0.201270 normal_loss: 0.103102
[011/00074] train_loss: 0.117859 kl_loss: 0.185326 normal_loss: 0.099327
[013/00024] train_loss: 0.112892 kl_loss: 0.167775 normal_loss: 0.096114
[014/00049] train_loss: 0.110428 kl_loss: 0.156501 normal_loss: 0.094778
[015/00074] train_loss: 0.106607 kl_loss: 0.147298 normal_loss: 0.091878
[017/00024] train_loss: 0.102991 kl_loss: 0.138078 normal_loss: 0.0

In [217]:
#####################
#                   #
#    VISUALIZING    #
#                   #
#####################

In [155]:
from scripts.visualize import visualize_dataset_sample, visualize_ad, visualize_vad, visualize_vad_norm, visualize_vad_norm

In [245]:
import random
experiment = "airplane_vad"
experiment2 = "sofa_ad"
# experiment2 = "sofa_ad"
filter_class = "airplane"
index = 4123
index1 = random.choice(range(len(ShapeNet('train', filter_class = "airplane"))))
index2 = random.choice(range(len(ShapeNet('train', filter_class = filter_class))))
a1 = 0.5
a2 = 1 - a1
#-------
# visualize_ad("airplane_ad", index1)
#-------
visualize_vad_norm("chair_vad_0.1kl")
# visualize_vad_norm(experiment)
# visualize_ad(experiment, index1)
# visualize_vad_norm(experiment2)
# visualize_ad(experiment, index)
#-------
# visualize_vad_norm(experiment)
# visualize_vad_norm(experiment2)
# visualize_dataset_sample(filter_class, index)
#-------
# visualize_interpolation_ad(experiment, index1, index2, a1, a2)
# visualize_ad(experiment, index1)
# visualize_ad(experiment, index2)

Output()

In [168]:
####################
#                  #
#    EVALUATION    #
#                  #
####################

In [169]:
from scripts.evaluate import generate_samples, convert_df_to_point_cloud, chamfer_distance, MMD, convert_set_to_point_cloud, visualize_point_cloud, _mmd, TMD, IOU
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes
import numpy as np
import random

In [241]:
# TMD
tmd, samples = TMD('chair_vad_0.1kl', n_samples=10, device=torch.device('cuda:0'))

In [242]:
tmd

tensor(0.0606, device='cuda:0')

In [203]:
index = 8
input_mesh = marching_cubes(samples[index].cpu().detach().numpy(), level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

Output()

In [233]:
# MMD
mmd, mmds = MMD('chair_vad_0.1kl', 'val', 'chair', n_samples=10, device=torch.device('cuda:0'))

In [234]:
mmd

tensor(0.0053, device='cuda:0')

In [7]:
# generate n new samples
n = 10
samples = generate_samples('table_vad', n)
samples = samples.squeeze(1)
# convert_set_to_point_cloud(samples)

In [119]:
index = 8
input_mesh = marching_cubes(samples[index].detach().numpy(), level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

Output()

In [9]:
val = []
val_dataset = ShapeNet('val', filter_class='airplane')
for data_dict in val_dataset:
    target_df = torch.from_numpy(data_dict['target_df']).float()
    val.append(target_df)
val = torch.stack(val)

In [19]:
index = 0
# visualize
input_mesh = marching_cubes(val[index].detach().numpy(), level=1)
input_mesh2 = marching_cubes(samples[index].detach().numpy(), level=1)
# visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)
point_cloud = convert_df_to_point_cloud(val[index])
point_cloud2 = convert_df_to_point_cloud(samples[index])
print(point_cloud)
print(point_cloud2)
visualize_point_cloud(point_cloud.numpy())
visualize_point_cloud(point_cloud2.numpy())

tensor([[-0.1278, -0.0485,  0.0794],
        [ 0.5927,  0.0102, -0.0656],
        [-0.2694, -0.2313, -0.0475],
        ...,
        [ 0.2366, -0.2002, -0.0627],
        [-0.2262, -0.0294,  0.4132],
        [-0.3590, -0.0955, -0.7992]])
tensor([[ 0.1878, -0.1604, -0.5195],
        [ 0.7181, -0.0691, -0.2029],
        [ 0.0807, -0.2327, -0.4569],
        ...,
        [-0.0027,  0.0769,  0.7589],
        [-0.0543, -0.1888,  0.0906],
        [-0.0586, -0.1129, -0.2101]])


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [10]:
samples_point_clouds = convert_set_to_point_cloud(val[0:10], count=10000)
samples_point_clouds2 = convert_set_to_point_cloud(val[0:1], count=10000)

In [11]:
samples_point_clouds[0].shape

torch.Size([10000, 3])

In [63]:
mmd_value = _mmd(samples_point_clouds, samples_point_clouds2)

1: 0.00033399579115211964


In [38]:
_mmd(samples, samples)

ValueError: too many values to unpack (expected 2)

In [33]:
cf = chamfer_distance(point_cloud, point_cloud2)

tensor([0.1142, 0.0598, 0.0302,  ..., 0.0104, 0.0498, 0.0848])
tensor([0.0288, 0.0441, 0.0509,  ..., 0.0924, 0.1467, 0.1362])


In [34]:
print(cf, cf / 2)

tensor(0.1314) tensor(0.0657)


In [23]:
print(point_cloud.shape)

torch.Size([2048, 3])


In [31]:
min = 10000
for i in range(2048):
    dist = torch.linalg.norm(point_cloud[5]-point_cloud2[i], axis=0)
    if dist < min:
        min = dist
print(min)


tensor(0.0521)


In [21]:
torch.linalg.norm(point_cloud[0]-point_cloud2[0], axis=0) # num_points * num_points

tensor(0.6862)

In [41]:
# MMD
mmd, mmds = MMD('airplane_vad', 'val', 'airplane', n_samples=10, device=torch.device('cuda:0'))

1: 0.18461653590202332
2: 0.18126583099365234
3: 0.18387258052825928
4: 0.18040665984153748
5: 0.1862756311893463
6: 0.20319686830043793
7: 0.16356226801872253
8: 0.1683862805366516
9: 0.16741611063480377
10: 0.17503449320793152


In [17]:
mmd

tensor(0.1371, device='cuda:0')